<center><img src='../../img/ai4eo_logos.jpg' alt='Logos AI4EO MOOC' width='80%'></img></center>

<hr>

<br>

<a href="https://www.futurelearn.com/courses/artificial-intelligence-for-earth-monitoring/1/steps/1280514"><< Back to FutureLearn</a><br>

# 3B - Tile-based classification using Sentinel-2 L1C and EuroSAT data - Functions

<i>by Nicolò Taggio, Planetek Italia S.r.l., Bari, Italy</i>

<hr>

### <a id='from_folder_to_stack'></a> `from_folder_to_stack`

In [2]:
'''
function name: 
    from_folder_to_stack
description:
    This function transform the .SAFE file into three different arrays (10m, 20m and 60m).
Input:
    safe_path: the path of the .SAFE file;
    data_bands_20m: if True, the function computes stack using Sentinel2 band with 20m of pixel resolution (default=True);
    data_bands_60m: if True, the function computes stack using Sentinel2 band with 60m of pixel resolution (default=True);    
Output: 
    stack_10m: stack with the following S2L1C bands (B02,B03,B04,B08)
    stack_20m: stack with the following S2L1C bands (B05,B06,B07,B11,B12,B8A)
    stack_60m: stack with the following S2L1C bands (B01,B09,B10)
'''
def from_folder_to_stack(
        safe_path,
        data_bands_20m=True,
        data_bands_60m=True,
        ):        
    
    level_folder_name_list = glob.glob(safe_path + 'GRANULE/*') 
    level_folder_name = level_folder_name_list[0]
    
    if level_folder_name.find("L2A") < 0:
        safe_path = [level_folder_name + '/IMG_DATA/']
    else:
        safe_path_10m = level_folder_name + '/IMG_DATA/R10m/'
        safe_path = [safe_path_10m]
    
    text_files = []

    for i in range(0,len(safe_path)):
        print("[AI4EO_MOOC]_log: Loading .jp2 images in %s" % (safe_path[i]))
        text_files_tmp = [f for f in os.listdir(safe_path[i]) if f.endswith('.jp2')]
        text_files.append(text_files_tmp)
        
    lst_stack_60m=[]
    lst_code_60m =[]
    lst_stack_20m=[]
    lst_code_20m =[]
    lst_stack_10m=[]
    lst_code_10m =[]
    for i in range(0,len(safe_path)):        
        
        print("[AI4EO_MOOC]_log: Reading .jp2 files in %s" % (safe_path[i]))
        for name in range(0, len(text_files[i])):            
            text_files_tmp = text_files[i]               
            if data_bands_60m == True:
                cond_60m = ( (text_files_tmp[name].find("B01") > 0) or (text_files_tmp[name].find("B09") > 0) 
                            or (text_files_tmp[name].find("B10") > 0))
                if cond_60m:
                    print("[AI4EO_MOOC]_log: Using .jp2 image: %s" % text_files_tmp[name])
                    lst_stack_60m.append(gdal_array.LoadFile(safe_path[i] + text_files_tmp[name]))
                    lst_code_60m.append(text_files_tmp[name][24:26])
                
            if data_bands_20m == True:                    
                cond_20m = (text_files_tmp[name].find("B05") > 0) or (text_files_tmp[name].find("B06") > 0) or (
                            text_files_tmp[name].find("B07") > 0) or (text_files_tmp[name].find("B11") > 0) or (
                                       text_files_tmp[name].find("B12") > 0) or (text_files_tmp[name].find("B8A") > 0)
                cond_60m_L2 = (text_files_tmp[name].find("B05_60m") < 0) and (text_files_tmp[name].find("B06_60m") < 0) and (
                            text_files_tmp[name].find("B07_60m") < 0) and (text_files_tmp[name].find("B11_60m") < 0) and (
                                       text_files_tmp[name].find("B12_60m") < 0) and (text_files_tmp[name].find("B8A_60m") < 0)
                cond_20m_tot = cond_20m and cond_60m_L2
                if cond_20m_tot:
                    print("[AI4EO_MOOC]_log: Using .jp2 image: %s" % text_files_tmp[name])
                    lst_stack_20m.append(gdal_array.LoadFile(safe_path[i] + text_files_tmp[name]))
                    lst_code_20m.append(text_files_tmp[name][24:26])
            else:
                stack_20m = 0
                    
            cond_10m = (text_files_tmp[name].find("B02") > 0) or (text_files_tmp[name].find("B03") > 0) or (
                        text_files_tmp[name].find("B04") > 0) or (text_files_tmp[name].find("B08") > 0)
            cond_20m_L2 = (text_files_tmp[name].find("B02_20m") < 0) and (text_files_tmp[name].find("B03_20m") < 0) and (
                        text_files_tmp[name].find("B04_20m") < 0) and (text_files_tmp[name].find("B08_20m") < 0)
            cond_60m_L2 = (text_files_tmp[name].find("B02_60m") < 0) and(text_files_tmp[name].find("B03_60m") < 0) and(
                        text_files_tmp[name].find("B04_60m") < 0) and (text_files_tmp[name].find("B08_60m") < 0)
            cond_10m_tot = cond_10m and cond_20m_L2 and cond_60m_L2
            
            if cond_10m_tot:
                print("[AI4EO_MOOC]_log: Using .jp2 image: %s" % text_files_tmp[name])
                lst_stack_10m.append(gdal_array.LoadFile(safe_path[i] + text_files_tmp[name]))
                lst_code_10m.append(text_files_tmp[name][24:26])
                 
    
    stack_10m=np.asarray(lst_stack_10m)
    sorted_list_10m = ['02','03','04','08']    
    print('[AI4EO_MOOC]_log: Sorting stack 10m...')
    stack_10m_final_sorted = stack_sort(stack_10m, lst_code_10m, sorted_list_10m)
    
    stack_20m=np.asarray(lst_stack_20m)
    sorted_list_20m = ['05','06','07','11','12','8A']
    print('[AI4EO_MOOC]_log: Sorting stack 20m...')
    stack_20m_final_sorted = stack_sort(stack_20m, lst_code_20m, sorted_list_20m)
    
    stack_60m=np.asarray(lst_stack_60m)
    sorted_list_60m = ['01','09','10']    
    print('[AI4EO_MOOC]_log: Sorting stack 60m...')
    stack_60m_final_sorted = stack_sort(stack_60m, lst_code_60m, sorted_list_60m)
                
    return stack_10m_final_sorted, stack_20m_final_sorted, stack_60m_final_sorted

<br>

### <a id='stack_sort'></a>`stack_sort`

In [ ]:
def stack_sort(stack_in, lst_code, sorted_list):    
    b,r,c = stack_in.shape
    stack_sorted = np.zeros((r,c,b), dtype=np.uint16)
    
    len_list_bands = len(lst_code)
    
    c = np.zeros((len_list_bands),dtype=np.uint8)
    count = 0
    count_sort = 0
    while count_sort != len_list_bands:
        if lst_code[count] == sorted_list[count_sort]:
            c[count_sort] = count
            count_sort = count_sort + 1
            count = 0
        else:
            count = count + 1   
    print('[AI4EO_MOOC]_log: sorted list:', sorted_list)
    print('[AI4EO_MOOC]_log: bands:', c)
    for i in range(0, len_list_bands):
        stack_sorted[:,:,i]=stack_in[c[i],:,:]
        
    return stack_sorted

<br>

### <a id='resample_3d'></a>`resample_3d`

In [ ]:
'''
function name: 
    resample_3d
description:
    Wrapper of ndimage zoom. Bilinear interpolation for resampling array
Input:
    stack: array to be resampled;
    row10m: the expected row;
    col10m: the expected col;
    rate: the rate of the tranformation;
Output: 
    stack_10m: resampled array
'''
def resample_3d(
        stack,
        row10m,
        col10m,
        rate):
    row, col, bands = stack.shape
    print("[AI4EO_MOOC]_log: Array shape (%d,%d,%d)" % (row, col, bands))
    
    stack_10m = np.zeros((row10m, col10m, bands),dtype=np.uint16)
    print("[AI4EO_MOOC]_log: Resize array bands from (%d,%d,%d) to (%d,%d,%d)" % (
        row, col, bands, row10m, col10m, bands))
    
    for i in range(0, bands):
        stack_10m[:, :, i] = ndimage.zoom(stack[:, :,i], rate)
        
    del (stack)
    
    return stack_10m

<br>

### <a id='sentinel2_format'></a>`sentinel2_format`

In [ ]:
'''
function name: 
    sentinel2_format
description:
    This function transform the multistack into sentinel2 format array with bands in the right position for AI model.
Input:
    total_stack: array that is the concatenation of stack10, stack_20mTo10m and stack_60mTo10m,;    
Output: 
    sentinel2: sentinel2 format array    
'''
def sentinel2_format(
        total_stack):
    
    row_tot, col_tot, bands_tot = total_stack.shape
    sentinel2 = np.zeros((row_tot, col_tot,bands_tot),dtype=np.uint16)   
    
    
    print("[AI4EO_MOOC]_log: Creating total stack with following bands list:")
    print("[AI4EO_MOOC]_log: Band 1 – Coastal aerosol")
    print("[AI4EO_MOOC]_log: Band 2 – Blue")
    print("[AI4EO_MOOC]_log: Band 3 – Green")
    print("[AI4EO_MOOC]_log: Band 4 – Red")
    print("[AI4EO_MOOC]_log: Band 5 – Vegetation red edge")
    print("[AI4EO_MOOC]_log: Band 6 – Vegetation red edge")
    print("[AI4EO_MOOC]_log: Band 7 – Vegetation red edge")
    print("[AI4EO_MOOC]_log: Band 8 – NIR")
    print("[AI4EO_MOOC]_log: Band 8A – Narrow NIR")
    print("[AI4EO_MOOC]_log: Band 9 – Water vapour")
    print("[AI4EO_MOOC]_log: Band 10 – SWIR – Cirrus")
    print("[AI4EO_MOOC]_log: Band 11 – SWIR")
    print("[AI4EO_MOOC]_log: Band 12 – SWIR")

    sentinel2[:, :, 0] = total_stack[:, :, 10]
    sentinel2[:, :, 1] = total_stack[:, :, 0]
    sentinel2[:, :, 2] = total_stack[:, :, 1]
    sentinel2[:, :, 3] = total_stack[:, :, 2]
    sentinel2[:, :, 4] = total_stack[:, :, 4]
    sentinel2[:, :, 5] = total_stack[:, :, 5]
    sentinel2[:, :, 6] = total_stack[:, :, 6]
    sentinel2[:, :, 7] = total_stack[:, :, 3]
    sentinel2[:, :, 8] = total_stack[:, :, 9]
    sentinel2[:, :, 9] = total_stack[:, :,11]
    sentinel2[:, :,10] = total_stack[:, :,12]
    sentinel2[:, :,11] = total_stack[:, :, 7]
    sentinel2[:, :,12] = total_stack[:, :, 8]
    
    
    
    del (total_stack)
    return sentinel2

<br>

## `sliding`

In [ ]:
'''
Function_name: 
    sliding
description: 
Input:
    shape: the target shape
    window_size: the shape of the window
    step_size:
    fixed
Output:
   windows:
'''


def sliding(shape, window_size, step_size=None, fixed=True):
    
    h, w = shape
    if step_size:
        h_step = step_size
        w_step = step_size
    else:
        h_step = window_size
        w_step = window_size
        
    h_wind = window_size
    w_wind = window_size
    windows = []
    for y in range(0, h, h_step):
        for x in range(0, w, w_step):
            h_min = min(h_wind, h - y)
            w_min = min(w_wind, w - x)
            if fixed:
                if h_min < h_wind or w_min < w_wind:
                    continue
            window = (x, y, w_min, h_min)
            windows.append(window)

    return windows

<br>

<a href="https://www.futurelearn.com/courses/artificial-intelligence-for-earth-monitoring/1/steps/1170903"><< Back to FutureLearn</a><br>

<hr>

<img src='../../img/copernicus_logo.png' alt='Copernicus logo' align='left' width='20%'></img>

Course developed for [EUMETSAT](https://www.eumetsat.int/), [ECMWF](https://www.ecmwf.int/) and [Mercator Ocean International](https://www.mercator-ocean.fr/en/) in support of the [EU’s Copernicus Programme](https://www.copernicus.eu/en) and the [WEkEO platform](https://wekeo.eu/).
